In [ ]:
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoTokenizer,AutoModel
from typing import List,Optional

In [ ]:
class ChatGLM2(LLM):
    max_token:int=4096  
    temperature:float=0.8  # 让词语多样性更高
    top_p=0.9       # 选择最大的概率之和小于0.9
    tokenizer:object=None
    model:object=None
    history=[]
    def __init_(self):
        super().__init__()
    
    @property
    def _llm_type(self)->str:
        return "ChatGLM2"
    
    def load_model(self, model_path=None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None):
        response, history = self.model.chat(self.tokenizer, prompt, history=self.history, max_length=self.max_token, temperature=self.temperature, top_p=self.top_p)
        
        # 这里如何设置流式输出是个问题
        
        if stop is not None:
            response = enforce_stop_tokens(response, stop)
        self.history = self.history+[[None, response]]
        return response
    
